In [1]:
import numpy as np

In [2]:
import pandas

In [3]:
import os

In [4]:
class FileIndex:
    def __init__(self, path='.', filter_function=None, debug=False):
        self._debug = debug
        self._filter_function = filter_function
        self._df = self._create_df_from_lst(list([self._get_lst_entry_from_path(entry=path)]) + 
                                            list(self._scandir(path=path, df=None, recursive=True)))

    def _init_df_lst(self, path_lst, df=None, include_root=True):
        total_lst = []
        for p in path_lst: 
            if include_root:
                total_lst.append(self._get_lst_entry_from_path(entry=p))
            for entry in list(self._scandir(path=p, df=df, recursive=True)):
                total_lst.append(entry)
        return self._create_df_from_lst(total_lst)
    
    def _scandir(self, path, df=None, recursive=True):
        if df is not None and len(df) > 0:
            for entry in os.scandir(path):
                if entry.path not in df.path.values:
                    if entry.is_dir(follow_symlinks=False) and recursive:
                        yield from self._scandir(path=entry.path, df=df, recursive=recursive)
                        yield self._get_lst_entry(entry=entry)
                    else:
                        yield self._get_lst_entry(entry=entry)
        else: 
            for entry in os.scandir(path):
                if entry.is_dir(follow_symlinks=False) and recursive:
                    yield from self._scandir(path=entry.path, recursive=recursive)
                    yield self._get_lst_entry(entry=entry)
                else:
                    yield self._get_lst_entry(entry=entry)
                    
    def _get_changes_quick(self):  # This one checks all files - it might be better to only check directories 
        path_exists_bool_lst = [os.path.exists(p) for p in self._df.path.values]
        path_deleted_lst = self._df[~np.array(path_exists_bool_lst)].path.values
        df_exists = self._df[path_exists_bool_lst]
        stat_lst = [os.stat(p) for p in df_exists.path.values]
        st_mtime = [s.st_mtime for s in stat_lst]
        st_nlink = [s.st_nlink for s in stat_lst]
        df_modified = df_exists[~np.isclose(df_exists.mtime.values, st_mtime, rtol=1e-10, atol=1e-15) | 
                                np.not_equal(df_exists.nlink.values, st_nlink)]
        if len(df_modified) > 0:
            if sum(df_modified.is_directory.values) > 0:
                dir_changed_lst = df_modified[df_modified.is_directory].path.values
            else: 
                dir_changed_lst = []
            files_changed_lst = df_modified.path.values
        else: 
            files_changed_lst, dir_changed_lst = [], []
        df_new = self._init_df_lst(path_lst=dir_changed_lst, df=df_exists, include_root=False)
        return df_new, files_changed_lst, path_deleted_lst
    
    def update(self):
        df_new, files_changed_lst, path_deleted_lst = self._get_changes_quick()
        if self._debug:
            print('Changes: ', df_new.path.values, files_changed_lst, path_deleted_lst)
        if len(path_deleted_lst) != 0:
            self._df = self._df[~self._df.path.isin(path_deleted_lst)]
        if len(files_changed_lst) != 0:
            df_updated = self._create_df_from_lst([self._get_lst_entry_from_path(entry=f) 
                                                   for f in files_changed_lst])
            self._df = self._df[~self._df.path.isin(df_updated.path)]
            self._df = pandas.concat([self._df, df_updated]).drop_duplicates().reset_index(drop=True)
        if len(df_new) != 0:
            self._df = pandas.concat([self._df, df_new]).drop_duplicates().reset_index(drop=True)
    
    def _repr_html_(self):
        return self._df._repr_html_()
    
    def _get_lst_entry_from_path(self, entry):
        stat = os.stat(entry)
        isdir = os.path.isdir(entry)
        if not isdir and self._filter_function is not None: 
            flag = self._filter_function(entry)
        else: 
            flag = True 
        if flag:
            return [os.path.basename(entry), 
                    entry, 
                    os.path.dirname(entry), 
                    isdir, 
                    stat.st_mtime, 
                    stat.st_nlink]
        else: 
            return []
    
    def _get_lst_entry(self, entry):
        stat = entry.stat()
        isdir = entry.is_dir()
        if not isdir and self._filter_function is not None: 
            flag = self._filter_function(entry.path)
        else: 
            flag = True
        if flag:
            return [entry.name, 
                    entry.path, 
                    os.path.dirname(entry.path), 
                    isdir, 
                    stat.st_mtime, 
                    stat.st_nlink]
        else: 
            return []
    
    @staticmethod
    def _create_df_from_lst(lst): 
        lst_clean = [l for l in lst if len(l) != 0]
        if len(lst_clean) != 0:
            name_lst, path_lst, dirname_lst, dir_lst, mtime_lst, nlink_lst = zip(*lst_clean)
        else: 
            name_lst, path_lst, dirname_lst, dir_lst, mtime_lst, nlink_lst  = [], [], [], [], [], []
        return pandas.DataFrame({'basename': name_lst, 
                                 'path': path_lst, 
                                 'dirname': dirname_lst, 
                                 'is_directory': dir_lst, 
                                 'mtime': mtime_lst, 
                                 'nlink': nlink_lst})



In [5]:
def touch(fname, times=None):
    with open(fname, 'a'):
        os.utime(fname, times)

In [6]:
def filter_function(file_name):
    return '.txt' in file_name

In [7]:
fi = FileIndex(path='.', filter_function=filter_function, debug=True)
fi

,basename,path,dirname,is_directory,mtime,nlink
0,.,.,,True,1.563191e+09,17
1,strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5/strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5,True,1.563103e+09,3
2,strain_0_98_hdf5,./demo/murn_hdf5/strain_0_98_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
3,strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5/strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5,True,1.563103e+09,3
4,strain_0_94_hdf5,./demo/murn_hdf5/strain_0_94_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
5,strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5/strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5,True,1.563103e+09,3
6,strain_1_0_hdf5,./demo/murn_hdf5/strain_1_0_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
7,strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5/strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5,True,1.563103e+09,3
8,strain_0_92_hdf5,./demo/murn_hdf5/strain_0_92_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
9,murn,./demo/murn_hdf5/murn,./demo/murn_hdf5,True,1.563103e+09,3


In [8]:
fi.update()
fi

Changes:  [] [] []


,basename,path,dirname,is_directory,mtime,nlink
0,.,.,,True,1.563191e+09,17
1,strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5/strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5,True,1.563103e+09,3
2,strain_0_98_hdf5,./demo/murn_hdf5/strain_0_98_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
3,strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5/strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5,True,1.563103e+09,3
4,strain_0_94_hdf5,./demo/murn_hdf5/strain_0_94_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
5,strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5/strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5,True,1.563103e+09,3
6,strain_1_0_hdf5,./demo/murn_hdf5/strain_1_0_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
7,strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5/strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5,True,1.563103e+09,3
8,strain_0_92_hdf5,./demo/murn_hdf5/strain_0_92_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
9,murn,./demo/murn_hdf5/murn,./demo/murn_hdf5,True,1.563103e+09,3


In [9]:
os.makedirs('bla')

In [10]:
fi.update()
fi

Changes:  ['./bla'] ['.'] []


,basename,path,dirname,is_directory,mtime,nlink
0,strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5/strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5,True,1.563103e+09,3
1,strain_0_98_hdf5,./demo/murn_hdf5/strain_0_98_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
2,strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5/strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5,True,1.563103e+09,3
3,strain_0_94_hdf5,./demo/murn_hdf5/strain_0_94_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
4,strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5/strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5,True,1.563103e+09,3
5,strain_1_0_hdf5,./demo/murn_hdf5/strain_1_0_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
6,strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5/strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5,True,1.563103e+09,3
7,strain_0_92_hdf5,./demo/murn_hdf5/strain_0_92_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
8,murn,./demo/murn_hdf5/murn,./demo/murn_hdf5,True,1.563103e+09,3
9,strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5/strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5,True,1.563103e+09,3


In [11]:
os.stat('bla')

os.stat_result(st_mode=16877, st_ino=8641270648, st_dev=16777220, st_nlink=2, st_uid=501, st_gid=20, st_size=64, st_atime=1563263310, st_mtime=1563263310, st_ctime=1563263310)

In [12]:
os.makedirs('bla/bla')  # the only thing that changed is the st_nlink

In [13]:
fi.update()
fi

Changes:  ['./bla/bla'] ['./bla'] []


,basename,path,dirname,is_directory,mtime,nlink
0,strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5/strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5,True,1.563103e+09,3
1,strain_0_98_hdf5,./demo/murn_hdf5/strain_0_98_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
2,strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5/strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5,True,1.563103e+09,3
3,strain_0_94_hdf5,./demo/murn_hdf5/strain_0_94_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
4,strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5/strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5,True,1.563103e+09,3
5,strain_1_0_hdf5,./demo/murn_hdf5/strain_1_0_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
6,strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5/strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5,True,1.563103e+09,3
7,strain_0_92_hdf5,./demo/murn_hdf5/strain_0_92_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
8,murn,./demo/murn_hdf5/murn,./demo/murn_hdf5,True,1.563103e+09,3
9,strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5/strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5,True,1.563103e+09,3


In [14]:
os.stat('bla')  # if we wait long enough also st_mtime is updated 

os.stat_result(st_mode=16877, st_ino=8641270648, st_dev=16777220, st_nlink=3, st_uid=501, st_gid=20, st_size=96, st_atime=1563263310, st_mtime=1563263310, st_ctime=1563263310)

In [15]:
os.stat('bla/bla')

os.stat_result(st_mode=16877, st_ino=8641270652, st_dev=16777220, st_nlink=2, st_uid=501, st_gid=20, st_size=64, st_atime=1563263310, st_mtime=1563263310, st_ctime=1563263310)

In [16]:
touch('bla/bla/test.txt') # the only thing that changed is the st_nlink

In [17]:
fi.update()
fi

Changes:  ['./bla/bla/test.txt'] ['./bla/bla'] []


,basename,path,dirname,is_directory,mtime,nlink
0,strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5/strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5,True,1.563103e+09,3
1,strain_0_98_hdf5,./demo/murn_hdf5/strain_0_98_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
2,strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5/strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5,True,1.563103e+09,3
3,strain_0_94_hdf5,./demo/murn_hdf5/strain_0_94_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
4,strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5/strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5,True,1.563103e+09,3
5,strain_1_0_hdf5,./demo/murn_hdf5/strain_1_0_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
6,strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5/strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5,True,1.563103e+09,3
7,strain_0_92_hdf5,./demo/murn_hdf5/strain_0_92_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
8,murn,./demo/murn_hdf5/murn,./demo/murn_hdf5,True,1.563103e+09,3
9,strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5/strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5,True,1.563103e+09,3


In [18]:
os.stat('bla/bla') # if we wait long enough also st_mtime is updated 

os.stat_result(st_mode=16877, st_ino=8641270652, st_dev=16777220, st_nlink=3, st_uid=501, st_gid=20, st_size=96, st_atime=1563263311, st_mtime=1563263311, st_ctime=1563263311)

In [19]:
os.stat('bla')

os.stat_result(st_mode=16877, st_ino=8641270648, st_dev=16777220, st_nlink=3, st_uid=501, st_gid=20, st_size=96, st_atime=1563263310, st_mtime=1563263310, st_ctime=1563263310)

In [20]:
touch('bla/test.txt')

In [21]:
fi.update()
fi

Changes:  ['./bla/test.txt'] ['./bla'] []


,basename,path,dirname,is_directory,mtime,nlink
0,strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5/strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5,True,1.563103e+09,3
1,strain_0_98_hdf5,./demo/murn_hdf5/strain_0_98_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
2,strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5/strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5,True,1.563103e+09,3
3,strain_0_94_hdf5,./demo/murn_hdf5/strain_0_94_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
4,strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5/strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5,True,1.563103e+09,3
5,strain_1_0_hdf5,./demo/murn_hdf5/strain_1_0_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
6,strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5/strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5,True,1.563103e+09,3
7,strain_0_92_hdf5,./demo/murn_hdf5/strain_0_92_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
8,murn,./demo/murn_hdf5/murn,./demo/murn_hdf5,True,1.563103e+09,3
9,strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5/strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5,True,1.563103e+09,3


In [22]:
os.stat('bla')

os.stat_result(st_mode=16877, st_ino=8641270648, st_dev=16777220, st_nlink=4, st_uid=501, st_gid=20, st_size=128, st_atime=1563263311, st_mtime=1563263311, st_ctime=1563263311)

In [23]:
os.stat('bla/bla')

os.stat_result(st_mode=16877, st_ino=8641270652, st_dev=16777220, st_nlink=3, st_uid=501, st_gid=20, st_size=96, st_atime=1563263311, st_mtime=1563263311, st_ctime=1563263311)

In [24]:
touch('bla/bla/test.txt', (1330712280, 1330712292))  # if the file is modified only the file changes not the directory

In [25]:
fi.update()  # only the file changed not the directory! 
fi

Changes:  [] ['./bla/bla/test.txt'] []


,basename,path,dirname,is_directory,mtime,nlink
0,strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5/strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5,True,1.563103e+09,3
1,strain_0_98_hdf5,./demo/murn_hdf5/strain_0_98_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
2,strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5/strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5,True,1.563103e+09,3
3,strain_0_94_hdf5,./demo/murn_hdf5/strain_0_94_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
4,strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5/strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5,True,1.563103e+09,3
5,strain_1_0_hdf5,./demo/murn_hdf5/strain_1_0_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
6,strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5/strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5,True,1.563103e+09,3
7,strain_0_92_hdf5,./demo/murn_hdf5/strain_0_92_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
8,murn,./demo/murn_hdf5/murn,./demo/murn_hdf5,True,1.563103e+09,3
9,strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5/strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5,True,1.563103e+09,3


In [26]:
os.stat('bla/bla')

os.stat_result(st_mode=16877, st_ino=8641270652, st_dev=16777220, st_nlink=3, st_uid=501, st_gid=20, st_size=96, st_atime=1563263311, st_mtime=1563263311, st_ctime=1563263311)

In [27]:
os.remove('bla/bla/test.txt')  # if a file is deleted only the file changes not the directory 

In [28]:
fi.update()
fi

Changes:  [] ['./bla/bla'] ['./bla/bla/test.txt']


,basename,path,dirname,is_directory,mtime,nlink
0,strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5/strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5,True,1.563103e+09,3
1,strain_0_98_hdf5,./demo/murn_hdf5/strain_0_98_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
2,strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5/strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5,True,1.563103e+09,3
3,strain_0_94_hdf5,./demo/murn_hdf5/strain_0_94_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
4,strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5/strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5,True,1.563103e+09,3
5,strain_1_0_hdf5,./demo/murn_hdf5/strain_1_0_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
6,strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5/strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5,True,1.563103e+09,3
7,strain_0_92_hdf5,./demo/murn_hdf5/strain_0_92_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
8,murn,./demo/murn_hdf5/murn,./demo/murn_hdf5,True,1.563103e+09,3
9,strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5/strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5,True,1.563103e+09,3


In [29]:
os.stat('bla/bla')

os.stat_result(st_mode=16877, st_ino=8641270652, st_dev=16777220, st_nlink=2, st_uid=501, st_gid=20, st_size=64, st_atime=1563263311, st_mtime=1563263311, st_ctime=1563263311)

In [30]:
os.stat('bla')

os.stat_result(st_mode=16877, st_ino=8641270648, st_dev=16777220, st_nlink=4, st_uid=501, st_gid=20, st_size=128, st_atime=1563263311, st_mtime=1563263311, st_ctime=1563263311)

In [31]:
os.rmdir('bla/bla')

In [32]:
fi.update()
fi

Changes:  [] ['./bla'] ['./bla/bla']


,basename,path,dirname,is_directory,mtime,nlink
0,strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5/strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5,True,1.563103e+09,3
1,strain_0_98_hdf5,./demo/murn_hdf5/strain_0_98_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
2,strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5/strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5,True,1.563103e+09,3
3,strain_0_94_hdf5,./demo/murn_hdf5/strain_0_94_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
4,strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5/strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5,True,1.563103e+09,3
5,strain_1_0_hdf5,./demo/murn_hdf5/strain_1_0_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
6,strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5/strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5,True,1.563103e+09,3
7,strain_0_92_hdf5,./demo/murn_hdf5/strain_0_92_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
8,murn,./demo/murn_hdf5/murn,./demo/murn_hdf5,True,1.563103e+09,3
9,strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5/strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5,True,1.563103e+09,3


In [33]:
os.stat('bla')

os.stat_result(st_mode=16877, st_ino=8641270648, st_dev=16777220, st_nlink=3, st_uid=501, st_gid=20, st_size=96, st_atime=1563263311, st_mtime=1563263311, st_ctime=1563263311)

In [34]:
os.remove('bla/test.txt')

In [35]:
fi.update()
fi

Changes:  [] ['./bla'] ['./bla/test.txt']


,basename,path,dirname,is_directory,mtime,nlink
0,strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5/strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5,True,1.563103e+09,3
1,strain_0_98_hdf5,./demo/murn_hdf5/strain_0_98_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
2,strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5/strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5,True,1.563103e+09,3
3,strain_0_94_hdf5,./demo/murn_hdf5/strain_0_94_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
4,strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5/strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5,True,1.563103e+09,3
5,strain_1_0_hdf5,./demo/murn_hdf5/strain_1_0_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
6,strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5/strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5,True,1.563103e+09,3
7,strain_0_92_hdf5,./demo/murn_hdf5/strain_0_92_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
8,murn,./demo/murn_hdf5/murn,./demo/murn_hdf5,True,1.563103e+09,3
9,strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5/strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5,True,1.563103e+09,3


In [36]:
os.path.exists('bla/bla')

False

In [37]:
os.path.exists('bla')

True

In [38]:
os.rmdir('bla')

In [39]:
fi.update()
fi

Changes:  [] ['.'] ['./bla']


,basename,path,dirname,is_directory,mtime,nlink
0,strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5/strain_0_98,./demo/murn_hdf5/strain_0_98_hdf5,True,1.563103e+09,3
1,strain_0_98_hdf5,./demo/murn_hdf5/strain_0_98_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
2,strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5/strain_0_94,./demo/murn_hdf5/strain_0_94_hdf5,True,1.563103e+09,3
3,strain_0_94_hdf5,./demo/murn_hdf5/strain_0_94_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
4,strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5/strain_1_0,./demo/murn_hdf5/strain_1_0_hdf5,True,1.563103e+09,3
5,strain_1_0_hdf5,./demo/murn_hdf5/strain_1_0_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
6,strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5/strain_0_92,./demo/murn_hdf5/strain_0_92_hdf5,True,1.563103e+09,3
7,strain_0_92_hdf5,./demo/murn_hdf5/strain_0_92_hdf5,./demo/murn_hdf5,True,1.563103e+09,3
8,murn,./demo/murn_hdf5/murn,./demo/murn_hdf5,True,1.563103e+09,3
9,strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5/strain_0_9,./demo/murn_hdf5/strain_0_9_hdf5,True,1.563103e+09,3
